In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from tensorflow.keras.preprocessing import image
from pathlib import Path
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
p = Path('./train/')
dirs = p.glob('*')

In [7]:
for d in dirs:
    print(d)

train\Aerodactyl
train\Bulbasaur
train\Charmander
train\Dratini
train\Fearow
train\Meowth
train\Pikachu
train\Psyduck
train\Spearow
train\Squirtle


In [9]:
path = "./train/"

train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2, 
    horizontal_flip = True,
    fill_mode='nearest'
)

In [10]:
train_datagenerator = train_generator.flow_from_directory(
    path,
    target_size = (227,227),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1642 images belonging to 10 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense


In [15]:
model = Sequential([
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),
    MaxPooling2D((3, 3), strides=(2, 2)),
    
    Conv2D(256, (5, 5), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),
    
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes for the 10 Pokémon types
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_8 (Conv2D)           (None, 13, 13, 384)      

In [17]:
hist = model.fit(train_datagenerator, steps_per_epoch= train_datagenerator.samples // train_datagenerator.batch_size, epochs=50, verbose=1)

Epoch 1/50
 4/51 [=>............................] - ETA: 1:32 - loss: 22.0509 - accuracy: 0.1562

c:\Users\sanid\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


51/51 [==============================] - 77s 1s/step - loss: 3.8151 - accuracy: 0.1547
Epoch 2/50
51/51 [==============================] - 74s 1s/step - loss: 2.1263 - accuracy: 0.1876
Epoch 3/50
51/51 [==============================] - 75s 1s/step - loss: 1.9423 - accuracy: 0.2932
Epoch 4/50
51/51 [==============================] - 73s 1s/step - loss: 1.8501 - accuracy: 0.3112
Epoch 5/50
51/51 [==============================] - 74s 1s/step - loss: 1.7453 - accuracy: 0.3404
Epoch 6/50
51/51 [==============================] - 76s 1s/step - loss: 1.7115 - accuracy: 0.3348
Epoch 7/50
51/51 [==============================] - 75s 1s/step - loss: 1.5964 - accuracy: 0.3981
Epoch 8/50
51/51 [==============================] - 76s 1s/step - loss: 1.5830 - accuracy: 0.4112
Epoch 9/50
51/51 [==============================] - 77s 1s/step - loss: 1.5727 - accuracy: 0.4019
Epoch 10/50
51/51 [==============================] - 76s 1s/step - loss: 1.4915 - accuracy: 0.4267
Epoch 11/50
51/51 [===========

In [18]:
model.save("poekdex.h5")

In [ ]:
test_path = "./Test/images/"

validation_datagenerator = ImageDataGenerator(
    rescale=1./255
)

validationgen = validation_datagenerator.flow_from_directory(
    validationgen
)

In [16]:
import tensorflow as tf
print(f"Gpu available : {len(tf.config.experimental.list_physical_devices('GPU'))}")

Gpu available : 0


In [4]:
model_ = keras.models.load_model('poekdex.h5')

In [5]:
def image_processing(path):
    img = image.load_img(path, target_size=(227,227))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = img_arr/225.0

    return img_arr



In [6]:
def predict(model, path, classlabel):
    img_arr = image_processing(path)
    predictions = model.predict(img_arr)
    predicted_class = np.argmax(predictions, axis=1)[0]
    return classlabel[predicted_class]

classlabel = ["Aerodactyl","Bulbasaur","Charmander","Dratini","Fearow","Mewtwo","Pikachu","Psyduck","Spearow","Squirtle"]

In [7]:
predicted_class = predict(model_, "./Test/images/test_11.jpg", classlabel)

1/1 [==============================] - 0s 262ms/step


In [8]:
predicted_class

'Bulbasaur'

In [9]:
import os
test_path = "./Test/images/"
test_image_names = os.listdir("./Test/images/")

In [10]:
results = pd.DataFrame(columns=['Name','Class'])

In [11]:
test_path + test_image_names[1]

'./Test/images/test_1.jpg'

In [12]:
class_label_dict =   {'Aerodactyl': 0, 'Bulbasaur': 1, 'Charmander': 2, 'Dratini': 3, 'Fearow': 4, 'Mewtwo': 5, 'Pikachu': 6, 'Psyduck': 7, 'Spearow': 8, 'Squirtle': 9}

In [13]:
class_label_dict["Aerodactyl"]

0

In [14]:
for i in test_image_names:
    predicted_class = predict(model_, test_path + i, classlabel)
    results = results._append({'Name' : i, 'Class':class_label_dict[predicted_class] }, ignore_index=True)

1/1 [==============================] - 0s 80ms/step


In [15]:
results.to_csv('results2.csv', index=False)